# Phase 1: Planning & Model Setup

### Step 1: Load & Benchmark Model in Plaintext

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

model.eval()

prompt = "Explain the benefits of homomorphic encryption."
inputs = tokenizer(prompt, return_tensors="pt")

start = time.time()
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=32)
end = time.time()

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(f"Latency: {end - start:.2f} seconds")



/Users/rojo/Documents/repos/ZK-LLM-Turbo/split-inference-env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Explain the benefits of homomorphic encryption.
Latency: 0.66 seconds


### Step 2: Identify Linear vs Non-Linear Components

In [2]:
print(f"Latency: {end - start:.2f} seconds")
for name, module in model.named_modules():
    print(name, type(module))

Latency: 0.66 seconds
 <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>
model <class 'transformers.models.llama.modeling_llama.LlamaModel'>
model.embed_tokens <class 'torch.nn.modules.sparse.Embedding'>
model.layers <class 'torch.nn.modules.container.ModuleList'>
model.layers.0 <class 'transformers.models.llama.modeling_llama.LlamaDecoderLayer'>
model.layers.0.self_attn <class 'transformers.models.llama.modeling_llama.LlamaAttention'>
model.layers.0.self_attn.q_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.k_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.v_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.self_attn.o_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.mlp <class 'transformers.models.llama.modeling_llama.LlamaMLP'>
model.layers.0.mlp.gate_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.mlp.up_proj <class 'torch.nn.modules.linear.Linear'>
model.layers.0.mlp.down_proj <cla